Il package numpy contiene l'oggetto `nditer`, che viene usato per generare un iteratore su un `ndarray`. Ogni elemento è visitato quindi utilizzando l'interfaccia standard di Python "Iterator".

In [1]:
import numpy as np

a  = np.arange(0, 6).reshape(2, 3)
b = np.array([[5, 6,7], [8, 9, 0]], order="f")

print(a)

[[0 1 2]
 [3 4 5]]


In [2]:
for x in np.nditer(a):
    print(x, end=" ")
print("\n")
for x in np.nditer(b):
    print(x, end=" ")

0 1 2 3 4 5 

5 8 6 9 7 0 

Notiamo che nditer visita gli elementi in base a come sono memorizzati, quindi anche nel caso di trasposta in realtà l'ordine di visita non cambia:

In [3]:
print(a)
print(a.T)

for x in np.nditer(a, order="f"):
    print(x, end = " ")
    
print("\n")

for x in np.nditer(a.T):
    print(x, end=" ")

[[0 1 2]
 [3 4 5]]
[[0 3]
 [1 4]
 [2 5]]
0 3 1 4 2 5 

0 1 2 3 4 5 

Ricordiamo che l'ordinamento in memoria F (Fortran) è un ordinamento column-major, mentre l'ordinamento in memoria C è row-major. Possiamo passare un certo ordinamento anche ad nditer:

Di default un iteratore può solo leggere i valori di un nd-array, ma tramite un argomento chiamato `op_flags` possiamo modificare questo comportamento:

In [4]:
print(a)

with np.nditer(a, op_flags=['readwrite']) as it:
    for x in it:
        x[...] = 2*x
print(a)

[[0 1 2]
 [3 4 5]]
[[ 0  2  4]
 [ 6  8 10]]


Notiamo come usiamo il costrutto `with/as` di python, che ci consente di creare un context manager che si "auto-chiude" una volta terminate le operazioni specificate al suo interno (in questo caso il calcolo del doppio di ogni elemento). 

È possibile inoltre specificare una lista di flag. Ad esempio, `external_loop` esegue il loop non sul singolo elemento ma sulla struttura intera di più basso livello rispetto alla struttura originaria, risultando in un numero minore di iterazioni e quindi ad un'ottimizzazione migliore. 

In [5]:
print(a)

for x in np.nditer(a, flags=["external_loop"], order="F"):
    print(x)

[[ 0  2  4]
 [ 6  8 10]]
[0 6]
[2 8]
[ 4 10]


È possibile inoltre usare il flag "f_index", "c_index", "multi_index" (incompatibili con `external_loop`) per indicizzare gli elementi secondo la notazione fortran, c o multi_indice (una tupla contenente l'indirizzo dell'elemento)


In [6]:
it = np.nditer(a, flags=["multi_index"])
print(a)
for x in it:
    print(x, it.multi_index)

[[ 0  2  4]
 [ 6  8 10]]
0 (0, 0)
2 (0, 1)
4 (0, 2)
6 (1, 0)
8 (1, 1)
10 (1, 2)


Come abbiamo appena visto è possibile anche utilizzare una sintassi alternativa, che prevede l'assegnazione esplicita di un iteratore ad una variabile. Questa variabile può successivamente essere usata con un ciclo `while`:


In [7]:
it = np.nditer(a, flags=["c_index"])

while not it.finished:
    print("%d - %d" % (it[0], it.index))
    is_not_finished = it.iternext()

0 - 0
2 - 1
4 - 2
6 - 3
8 - 4
10 - 5


È possibile utilizzare un iteratore anche su array che sono `broadcastable`, ovvero che hanno shape diverse ma possono comunque essere utilizzati in operazioni matematico/aritmetiche per i quali apparentemente sarebbero incompatibili.

È sufficiente passare a nditer il _broadcast_ dei due ndarray: 
`[a, b, c, ..., N]`



In [8]:
a = np.arange(0,5)
b = np.arange(10).reshape(2, 5)

print(a)
print(b)

[0 1 2 3 4]
[[0 1 2 3 4]
 [5 6 7 8 9]]


In [14]:
for x, y in np.nditer([a, b]):
    print(a, b)

[0 1 2 3 4] [[0 1 2 3 4]
 [5 6 7 8 9]]
[0 1 2 3 4] [[0 1 2 3 4]
 [5 6 7 8 9]]
[0 1 2 3 4] [[0 1 2 3 4]
 [5 6 7 8 9]]
[0 1 2 3 4] [[0 1 2 3 4]
 [5 6 7 8 9]]
[0 1 2 3 4] [[0 1 2 3 4]
 [5 6 7 8 9]]
[0 1 2 3 4] [[0 1 2 3 4]
 [5 6 7 8 9]]
[0 1 2 3 4] [[0 1 2 3 4]
 [5 6 7 8 9]]
[0 1 2 3 4] [[0 1 2 3 4]
 [5 6 7 8 9]]
[0 1 2 3 4] [[0 1 2 3 4]
 [5 6 7 8 9]]
[0 1 2 3 4] [[0 1 2 3 4]
 [5 6 7 8 9]]
